In [ ]:
import csv

_FIELD_NAMES = [
    'song_title',
    'song_hash',
    'song_difficulty',
    'song_meter',
    'members_name',
    'score',
    'lamp'
]

song_hash_chars = {}

src = '3ic_data/3ic_data_20220805.csv'
with open(src, 'r', encoding='utf-8') as fp:
    reader = csv.DictReader(fp, fieldnames=_FIELD_NAMES)
    for row in reader:
        for c in row['song_hash']:
            if c in song_hash_chars:
                song_hash_chars[c] += 1
            else:
                song_hash_chars[c] = 0

print(len(song_hash_chars))

In [ ]:
for c in sorted(song_hash_chars):
    print(f'{c:3s} {ord(c):3d} 0x{ord(c):02X}')

In [ ]:
def a8e(s: str) -> str:
    s = s.strip()
    s2 = [c for c in s.lower() if (c >= 'a' and c <= 'z') or (c >= '0' and c <= '9')]
    l = len(s2)
    return f'{s[0]}{l-2}{s[-1]}'

for t in ['Puberty Dysthymia', 'Cytokinesis', 'Dance Dance Revolution']:
    print(a8e(t))


In [ ]:
v = [chr(v + ord('A')) for v in range(5)]

load_order = []

for i_row, a in enumerate(v):
    tri = v[i_row:]
    if i_row % 2 == 1:
         tri.reverse()
    for i_col, b in enumerate(tri):
        load_order.append( (a, b) )

load_replacement = [load_order[0]]
for prev, next in zip(load_order[:-1], load_order[1:]):
    repl = [a != b and b or None for a, b in zip(prev, next)]
    load_replacement.append(tuple(repl))

print(load_replacement)


In [ ]:
_PARTITION_SIZE = 3
_TOTAL_LENGTH = 26
_EXCLUDE_DIAGONAL = True
_EXPLICIT_PAIRS = False

v = [chr(v + ord('A')) for v in range(_TOTAL_LENGTH)]
v_part = [v[i:i+_PARTITION_SIZE] for i in range(0, _TOTAL_LENGTH, _PARTITION_SIZE)]
print(v_part)

load_order = []

load_replacement = []
exc_diag = _EXCLUDE_DIAGONAL and 1 or 0
for i_row in range(0, len(v_part), 2):
    # (i, i) --> (i, n)
    for i_col in range(i_row + exc_diag, len(v_part)):
        if _EXPLICIT_PAIRS or (i_col == i_row + exc_diag):
            load_replacement.append( (i_row, i_col) )
        else:
            load_replacement.append( (None, i_col) )

    # (i+1, i+1) <-- (i+1, n)
    if i_row + 1 >= len(v_part):
        break
    for i_col in range(len(v_part) - 1, i_row + exc_diag, -1):
        if i_col == len(v_part) - 1:
            load_replacement.append( (i_row + 1, _EXPLICIT_PAIRS and i_col or None) )
        else:
            load_replacement.append( (_EXPLICIT_PAIRS and (i_row + 1) or None, i_col) )

print(load_replacement)


In [ ]:
import requests
import logging
import os
import json
from datetime import datetime as dt

def timestamp():
    return dt.utcnow().strftime('%Y%m%d-%H%M%S-%f')[:-3]
p = os.path.join('itl2023_data', dt.utcnow().strftime('%Y%m%d'))

# Set up logging
logging.getLogger().handlers.clear()
log_stamp = timestamp()
log_path = os.path.join(p, f'scobility-scrape-{log_stamp}.log')
log_fmt = logging.Formatter(
    '[%(asctime)s.%(msecs)03d] %(levelname)-8s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logging.basicConfig(
    filename=log_path,
    encoding='utf-8',
    level=logging.INFO
)
logging.getLogger().addHandler(logging.StreamHandler())
for handler in logging.getLogger().handlers:
    handler.setFormatter(log_fmt)

In [ ]:
# Chart query
charts = {}
strikes = []
for i in range(10000):
    r = requests.get(f'https://itl2023.groovestats.com/api/chart/{i}')
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d}: {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        charts[i] = j.get('data', {})
        full_name = f"{charts[i].get('artist')} - \"{charts[i].get('title')}\""
        logging.info(f'{i:4d}: {full_name}')

if not os.path.exists(p):
    os.makedirs(p)
with open(os.path.join(p, 'charts.json'), 'w', encoding='utf-8') as fp:
    json.dump(charts, fp)

In [ ]:
p_entrants = os.path.join(p, 'entrant_info')
if not os.path.exists(p_entrants):
    os.makedirs(p_entrants)

# Entrant query
entrants = {}
strikes = []
for i in range(10000):
    r = requests.get(f'https://itl2023.groovestats.com/api/entrant/{i}')
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d}: {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        entrants[i] = j.get('data', {})
        full_name = f"{entrants[i]['entrant']['name']} (ITL #{entrants[i]['entrant']['id']}, GS #{entrants[i]['entrant']['membersId']})"
        logging.info(f'{i:4d}: {full_name}')

        with open(os.path.join(p_entrants, f'{i}.json'), 'w', encoding='utf-8') as fp:
            json.dump(entrants[i], fp)

In [ ]:
p_scores = os.path.join(p, 'song_scores')
if not os.path.exists(p_scores):
    os.makedirs(p_scores)

with open(os.path.join(p, 'charts.json'), 'r', encoding='utf-8') as fp:
    charts = json.load(fp)

# Entrant query
scores = {}
strikes = []
total = 0
for c in charts.values():
    total += 1
    if total > 10000:
        break

    i = c.get('id', 0)

    r = requests.post(
        f'https://itl2023.groovestats.com/api/score/chartTopScores',
        data={'chartHash': c['hash']}
    )
    j = r.json()
    if not j.get('success', False):
        logging.warning(f"{i:4d} (hash {c['hash']}): {j.get('message', '')}")
        strikes.append(i)
        if len(strikes) > 5:
            break
    else:
        strikes = []
        full_name = f"{c.get('artist')} - \"{c.get('title')}\""
        scores[i] = j.get('data', {}).get('leaderboard', {})
        for s in scores[i]:
            s['chartId'] = i
        logging.info(f"{i:4d} (hash {c['hash']}): {full_name}, {len(scores[i])} scores")

        with open(os.path.join(p_scores, f'{i}.json'), 'w', encoding='utf-8') as fp:
            json.dump({'scores': scores[i]}, fp)

In [ ]:
p_charts = os.path.join(p, 'song_info')
if not os.path.exists(p_charts):
    os.makedirs(p_charts)

with open(os.path.join(p, 'charts.json'), 'r', encoding='utf-8') as fp:
    charts = json.load(fp)

for c in charts.values():
    i = c.get('id', 0)
    
    full_name = f"{c.get('artist')} - \"{c.get('title')}\""
    logging.info(f"{i:4d} (hash {c['hash']}): {full_name}")

    with open(os.path.join(p_charts, f'{i}.json'), 'w', encoding='utf-8') as fp:
        json.dump({'song': c}, fp)